# Feature generation from mesh and fractures data

⚠️ **Important:** This notebook is **Step 2**. You must complete **Step 1** before running it.

---

### Pipeline Overview

1. **Step 1 — Run inside the PorePy environment**  
   **Input:** Model setup and fracture configuration.  
   **Output:**  
   - Mesh files: `mesh2d_Xfr_id.msh`  
   - Fracture metadata: `mesh_meta_id.json`  
   - Solver log: `solver_times.csv` (convergence, solve time, etc.)  

   **Example:**  
   `D:/MLpmg/Data_Large4` contains 4000 meshes, matching fracture files with IDs in `0.4d` format, and one `solver_times.csv`.

---

2. **Step 2 — Create the feature dataset** *(this notebook)*  
   **Input:**  
   - Mesh files `*.msh`  
   - Fracture metadata `*.json`  
   - Solver log `solver_times.csv`  

   **Output:**  
   - Feature table `mesh_features.csv` combining mesh geometry, fracture stats, and solver outcomes.  

   **Usage example:**
   ```python
   from dataset_builder import MeshFeatureDatasetBuilder

   builder = MeshFeatureDatasetBuilder(
       mesh_dir="D:/ML4pmg/Data_Large4/",
       solver_csv="D:/ML4pmg/Data_Large4/solver_times.csv",
       output_csv="D:/ML4pmg/Data_Large4/mesh_features.csv",
       log_transform=True
   )
   
---

3. **Step 3 – Run `Clf_and_Reg_Combined.ipynb`**  
   - Train models and generate predictions.  
   - Compare results against baseline and clairvoyant approaches.


In [ ]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import meshio
import json

from sklearn.decomposition import PCA
import seaborn as sns
from scipy.spatial import cKDTree


In [71]:
df = pd.read_csv("D:/ML4pmg/Data_Large4/solver_times.csv") # Data_Large, or Data_Large2
df.tail()

,Mesh #,Fractures #,Combination #,KSP Type,PC Mass,PC Interface,Converged,Krylov Iters,Construction Time (s),Solve Time (s),Total Time (s),Tic Toc Time (s),Solver Label,Adjusted Time (s)
71995,3999,49,13,tfqmr,sor,ilu,False,NaN,NaN,NaN,NaN,NaN,tfqmr + sor + ilu,4.850165
71996,3999,49,14,tfqmr,jacobi,ilu,False,NaN,NaN,NaN,NaN,NaN,tfqmr + jacobi + ilu,4.850165
71997,3999,49,15,tfqmr,bjacobi,ilu,False,NaN,NaN,NaN,NaN,NaN,tfqmr + bjacobi + ilu,4.850165
71998,3999,49,16,tfqmr,asm,ilu,False,NaN,NaN,NaN,NaN,NaN,tfqmr + asm + ilu,4.850165
71999,3999,49,17,tfqmr,gamg,ilu,True,180.0,0.149009,0.887573,1.036582,5.668525,tfqmr + gamg + ilu,1.036582


In [72]:
len(df)/18

4000.0

### Help Functions

In [73]:

def compute_edge_lengths(p0, p1, p2):
    a = np.linalg.norm(p1 - p0)
    b = np.linalg.norm(p2 - p1)
    c = np.linalg.norm(p0 - p2)
    return a, b, c

def compute_signed_area(p0, p1, p2):
    return 0.5 * ((p1[0] - p0[0]) * (p2[1] - p0[1]) - (p2[0] - p0[0]) * (p1[1] - p0[1]))

def compute_rms_edge(a, b, c):
    return np.sqrt((a ** 2 + b ** 2 + c ** 2) / 3)

def compute_internal_angles(p0, p1, p2):
    def angle(a, b, c):
        ba = a - b
        bc = c - b
        cosa = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosa, -1, 1)))
    return angle(p1, p0, p2), angle(p0, p1, p2), angle(p0, p2, p1)

def compute_triangle_center(p0, p1, p2):
    return (p0 + p1 + p2) / 3

def compute_volume_length_metric(signed_area, rms_edge):
    return (4 * np.sqrt(3) / 3) * signed_area / (rms_edge ** 2) if rms_edge > 0 else 0

def compute_triangle_features(p0, p1, p2):
    a, b, c = compute_edge_lengths(p0, p1, p2)
    signed_area = compute_signed_area(p0, p1, p2)
    area = abs(signed_area)
    rms_edge = compute_rms_edge(a, b, c)
    vol_length = compute_volume_length_metric(signed_area, rms_edge)
    angles = compute_internal_angles(p0, p1, p2)
    center = compute_triangle_center(p0, p1, p2)

    return {
        "area": area,
        "signed_area": signed_area,
        "rms_edge": rms_edge,
        "volume_length_metric": vol_length,
        "min_angle": min(angles),
        "max_angle": max(angles),
        "center": center
    }



# 📊 Feature Summary for Mesh and Fracture-Based Dataset

All array-based features are summarized with the following statistics:
- `mean`
- `median`
- `std`
- `min`, `max`
- `5pct`, `95pct`

The corresponding column names follow the pattern:  
`<feature>_<stat>`  
e.g., `rms_edge_mean`, `length_95pct`, etc.

---

## 🧱 Mesh-Based Features

| Feature Group            | Description                                          |
|--------------------------|------------------------------------------------------|
| `area_*`                 | Triangle area                                        |
| `signed_area_*`          | Signed triangle area (for orientation checks)        |
| `rms_edge_*`             | Root-mean-square of triangle edge lengths            |
| `volume_length_metric_*` | Shape quality metric: volume-to-length ratio         |
| `min_angle_*`            | Minimum internal angle per triangle                  |
| `max_angle_*`            | Maximum internal angle per triangle                  |
| `grading_ratio_*`        | Ratio of neighboring triangle sizes (element grading)|
| `n_elements`             | Total number of triangles in the mesh                |
| `n_nodes`                | Total number of mesh vertices                        |
| `n_fractures`            | Number of fractures in the mesh (from metadata or CSV)|

---

## 🌐 Fracture-Based Features

| Feature Group              | Description                                               |
|----------------------------|-----------------------------------------------------------|
| `length_*`                 | Lengths of all fractures                                  |
| `orientation_*`            | Orientation angles of fractures (in radians)              |
| `domain_span_x`            | Total horizontal extent (x-range) of all fracture endpoints |
| `domain_span_y`            | Total vertical extent (y-range) of all fracture endpoints |
| `fracture_density`         | Total fracture length divided by domain area              |
| `n_fractures`              | Number of fractures in the domain                         |
| `mortar_cells_2d_1d`       | Number of mortar cells between matrix (2D) and fractures (1D) |
| `mortar_cells_1d_0d`       | Number of mortar cells at fracture tips (1D–0D)           |
| `mortar_cells_total`       | Total mortar cells (`2d-1d` + `1d-0d`)                    |
| `mortar_cells_per_fracture`| Average number of 2d–1d mortar cells per fracture         |



### Main Class to build the dataset

Loop over all meshes in the folder and create the DataFrame of features, add labeles and number of fractures from solve_times.csv

In [74]:
class MeshFeatureDatasetBuilder:
    def __init__(self, mesh_dir, solver_csv, output_csv, log_transform=True):
        self.mesh_dir = mesh_dir
        self.solver_df = pd.read_csv(solver_csv)
        self.output_csv = output_csv
        self.log_transform = log_transform

        self.meta_dir = mesh_dir  # NEW
    
        
    

    def prepare_solver_tables(self):
        self.solver_times = self.solver_df.pivot_table(
            index='Mesh #', columns='Combination #', values='Adjusted Time (s)'
        ).sort_index()

        self.solver_convergence = self.solver_df.pivot_table(
            index='Mesh #', columns='Combination #', values='Converged'
        ).sort_index()

        self.fracture_counts = self.solver_df.groupby('Mesh #')['Fractures #'].first().to_dict()

    def summarize_array(self, arr, name):
        return {
            f"{name}_mean": np.mean(arr),
            f"{name}_median": np.median(arr),
            f"{name}_std": np.std(arr),
            f"{name}_5pct": np.percentile(arr, 5),
            f"{name}_95pct": np.percentile(arr, 95),
            f"{name}_min": np.min(arr),
            f"{name}_max": np.max(arr)
        }

    def compute_features_for_mesh(self, mesh_path, mesh_id):
        mesh = meshio.read(mesh_path)
        coords = mesh.points[:, :2]
        triangles = mesh.cells_dict.get("triangle")
        if triangles is None:
            raise ValueError(f"No triangles in {mesh_path}")

        areas, signed_areas, rms_edges, vol_lengths, min_angles, max_angles, centers = [], [], [], [], [], [], []
        for tri in triangles:
            pts = [coords[i] for i in tri]
            feats = compute_triangle_features(*pts)
            areas.append(feats["area"])
            signed_areas.append(feats["signed_area"])
            rms_edges.append(feats["rms_edge"])
            vol_lengths.append(feats["volume_length_metric"])
            min_angles.append(feats["min_angle"])
            max_angles.append(feats["max_angle"])
            centers.append(feats["center"])

        # Grading ratios
        tree = cKDTree(centers)
        grading_ratios = []
        for i, center in enumerate(centers):
            _, idxs = tree.query(center, k=2)
            h1, h2 = rms_edges[i], rms_edges[idxs[1]]
            grading_ratios.append(max(h1 / h2, h2 / h1))

        feature_dict = {
            "mesh_name": os.path.basename(mesh_path),
            "mesh_id": mesh_id,
            "n_elements": len(triangles),
            "n_nodes": len(coords),
            "n_fractures": self.fracture_counts.get(mesh_id, -1)
        }

        for arr, name in [
            (areas, "area"),
            (signed_areas, "signed_area"),
            (rms_edges, "rms_edge"),
            (vol_lengths, "volume_length_metric"),
            (min_angles, "min_angle"),
            (max_angles, "max_angle"),
            (grading_ratios, "grading_ratio")
        ]:
            feature_dict.update(self.summarize_array(arr, name))

        return feature_dict

    def extract_fracture_features(self, mesh_id):
        json_path = os.path.join(self.meta_dir, f"mesh_meta_{mesh_id:04d}.json")
        
        with open(json_path) as f:
            meta = json.load(f)

        fracs = meta["fractures"]
        n_fractures = len(fracs)
        lengths, orientations = [], []

        for frac in fracs:
            x0, y0 = frac[0]
            x1, y1 = frac[1]
            dx, dy = x1 - x0, y1 - y0
            lengths.append(np.hypot(dx, dy))
            orientations.append(np.arctan2(dy, dx))

        bbox = np.array(fracs).reshape(-1, 2)
        x_vals, y_vals = bbox[:, 0], bbox[:, 1]
        span_x = x_vals.max() - x_vals.min()
        span_y = y_vals.max() - y_vals.min()
        area = span_x * span_y
        total_length = np.sum(lengths)

        feat = {
            "n_fractures": n_fractures,
            "domain_span_x": span_x,
            "domain_span_y": span_y,
            "fracture_density": total_length / area if area > 0 else 0,
            "mortar_cells_2d_1d": meta["n_mortar_cells"]["2d-1d"],
            "mortar_cells_1d_0d": meta["n_mortar_cells"]["1d-0d"],
            "mortar_cells_total": meta["n_mortar_cells"]["2d-1d"] + meta["n_mortar_cells"]["1d-0d"],
            "mortar_cells_per_fracture": meta["n_mortar_cells"]["2d-1d"] / n_fractures if n_fractures > 0 else 0,
        }

        feat.update(self.summarize_array(lengths, "length"))
        feat.update(self.summarize_array(orientations, "orientation"))
        return feat



    def extract_features_for_mesh(self, mesh_id):
        mesh_path = os.path.join(self.mesh_dir, f"mesh2d_Xfr_{mesh_id:04d}.msh")
        feature_dict = self.compute_features_for_mesh(mesh_path, mesh_id)

        y_time = self.solver_times.loc[mesh_id].values
        y_conv = self.solver_convergence.loc[mesh_id].values.astype(float)

        if self.log_transform:
            y_time = -np.log(np.abs(y_time) + 1e-6)

        for i, t in enumerate(y_time):
            feature_dict[f"y_time_{i}"] = t
        for i, c in enumerate(y_conv):
            feature_dict[f"y_conv_{i}"] = c

        # Add fracture features from metadata
        if self.meta_dir is not None:
            try:
                fracture_features = self.extract_fracture_features(mesh_id)
                feature_dict.update(fracture_features)
            except Exception as e:
                print(f"⚠️ Failed to extract fracture features for mesh {mesh_id}: {e}")
        return feature_dict      

    def build_dataset(self):
        self.prepare_solver_tables()
        all_data = []
        for mesh_id in self.solver_times.index:  
#        for mesh_id in self.solver_times.index:
            try:
                feats = self.extract_features_for_mesh(mesh_id)
                all_data.append(feats)
                print(f"✅ Processed mesh {mesh_id}")
            except Exception as e:
                print(f"❌ Failed mesh {mesh_id}: {e}")

        df = pd.DataFrame(all_data)
        df.to_csv(self.output_csv, index=False)
        print(f"\n✅ Saved dataset to {self.output_csv}")


In [75]:
builder = MeshFeatureDatasetBuilder(
    mesh_dir="D:/ML4pmg/Data_Large4/",
    solver_csv="D:/ML4pmg/Data_Large4/solver_times.csv",
    output_csv="D:/ML4pmg/Data_Large4/mesh_features.csv",
    log_transform=True
)

t0 = time.time()
builder.build_dataset()
t1 = time.time()
print(f"Dataset built in {t1 - t0:.2f} seconds")


✅ Processed mesh 0

✅ Processed mesh 1

✅ Processed mesh 2

✅ Processed mesh 3

✅ Processed mesh 4

✅ Processed mesh 5

✅ Processed mesh 6

✅ Processed mesh 7

✅ Processed mesh 8

✅ Processed mesh 9

✅ Processed mesh 10

✅ Processed mesh 11

✅ Processed mesh 12

✅ Processed mesh 13

✅ Processed mesh 14

✅ Processed mesh 15

✅ Processed mesh 16

✅ Processed mesh 17

✅ Processed mesh 18

✅ Processed mesh 19

✅ Processed mesh 20

✅ Processed mesh 21

✅ Processed mesh 22

✅ Processed mesh 23

✅ Processed mesh 24

✅ Processed mesh 25

✅ Processed mesh 26

✅ Processed mesh 27

✅ Processed mesh 28

✅ Processed mesh 29

✅ Processed mesh 30

✅ Processed mesh 31

✅ Processed mesh 32

✅ Processed mesh 33

✅ Processed mesh 34

✅ Processed mesh 35

✅ Processed mesh 36

✅ Processed mesh 37

✅ Processed mesh 38

✅ Processed mesh 39

✅ Processed mesh 40

✅ Processed mesh 41

✅ Processed mesh 42

✅ Processed mesh 43

✅ Processed mesh 44

✅ Processed mesh 45

✅ Processed mesh 46

✅ Processed mesh 47

✅

In [77]:
(t1-t0)/ 60 /60 # Convert to hours


4.263296890788608

In [78]:
df = pd.read_csv("D:/ML4pmg/Data_Large4/mesh_features.csv")

In [79]:
df.describe()

,mesh_id,n_elements,n_nodes,n_fractures,area_mean,area_median,area_std,area_5pct,area_95pct,area_min,...,length_95pct,length_min,length_max,orientation_mean,orientation_median,orientation_std,orientation_5pct,orientation_95pct,orientation_min,orientation_max
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4.000000e+03,4000.000000,4.000000e+03,4000.000000,4.000000e+03,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,1999.500000,18521.958250,9363.844250,50.055250,0.000060,2.141441e-05,0.000110,3.103453e-07,0.000244,6.089272e-09,...,1.853203,0.081403,3.989039,0.344077,0.951395,2.005502,-2.826083,2.808525,-3.043726,3.043652
std,1154.844867,6435.699304,3227.146742,6.117398,0.000020,1.125918e-05,0.000030,4.116788e-07,0.000071,1.911691e-08,...,0.500347,0.044172,2.419019,0.294489,0.358863,0.125704,0.143715,0.152729,0.090784,0.094755
min,0.000000,6250.000000,3202.000000,40.000000,0.000009,7.507928e-09,0.000037,1.249292e-11,0.000045,3.548256e-17,...,1.033565,0.001228,1.130070,-0.754371,-1.834667,1.360317,-3.123601,1.969846,-3.141499,2.317778
25%,999.750000,14018.750000,7114.750000,45.000000,0.000046,1.372020e-05,0.000087,7.092088e-08,0.000192,1.463179e-10,...,1.492572,0.048746,2.500382,0.143799,0.811139,1.930246,-2.930814,2.715674,-3.112460,3.003127
50%,1999.500000,17805.500000,9000.000000,50.000000,0.000056,1.929655e-05,0.000105,1.768420e-07,0.000231,9.420736e-10,...,1.743742,0.074667,3.352428,0.344516,0.976157,2.014918,-2.841455,2.826385,-3.070549,3.071396
75%,2999.250000,21766.500000,10991.250000,55.000000,0.000071,2.692520e-05,0.000128,3.891183e-07,0.000288,4.258929e-09,...,2.092117,0.107894,4.711690,0.546078,1.152465,2.093026,-2.738223,2.921292,-3.002360,3.112218
max,3999.000000,112822.000000,56528.000000,60.000000,0.000160,9.649305e-05,0.000259,7.105846e-06,0.000563,4.165506e-07,...,5.030458,0.323586,37.347434,1.417098,1.960651,2.361802,-1.993027,3.125465,-2.586076,3.141591


### Update with Convergence Patterns and Pattern IDs


In [80]:
def annotate_with_convergence_patterns(dataset_csv, output_csv):
    df = pd.read_csv(dataset_csv)

    # Extract convergence columns (assumed to start with 'y_conv_')
    y_conv_cols = [col for col in df.columns if col.startswith('y_conv_')]
    if not y_conv_cols:
        raise ValueError("No convergence columns (y_conv_*) found in dataset!")

    # Step 1 — Build convergence pattern string
    df['convergence_pattern'] = df[y_conv_cols].astype(int).astype(str).agg(''.join, axis=1)

    # Step 2 — Count frequencies of unique patterns
    pattern_counts = df['convergence_pattern'].value_counts()

    # Step 3 — Assign pattern_id by frequency (most common is 0)
    pattern_to_id = {pattern: idx for idx, pattern in enumerate(pattern_counts.index)}
    df['pattern_id'] = df['convergence_pattern'].map(pattern_to_id)

    # Step 4 — Save updated dataset
    df.to_csv(output_csv, index=False)
    print(f"✅ Annotated dataset saved to {output_csv}")
    print("\nTop patterns with counts:")
    print(pattern_counts.head(10))

    return df, pattern_counts


In [81]:
updated_df, pattern_counts = annotate_with_convergence_patterns(
    dataset_csv="D:/ML4pmg/Data_Large4/mesh_features.csv",
    output_csv="D:/ML4pmg/Data_Large4/mesh_features_annotated.csv"
)


✅ Annotated dataset saved to D:/ML4pmg/Data_Large4/mesh_features_annotated.csv

Top patterns with counts:
convergence_pattern
000000000000000000    1400
000000100111000001     761
000000000001000001     591
000001100111000001     411
000000100110000000     276
000001100111100111     209
000000100111100111      93
000000000001000000      90
000001000001000001      62
000000100111000000      33
Name: count, dtype: int64


In [82]:
print(pattern_counts.head(15))

convergence_pattern
000000000000000000    1400
000000100111000001     761
000000000001000001     591
000001100111000001     411
000000100110000000     276
000001100111100111     209
000000100111100111      93
000000000001000000      90
000001000001000001      62
000000100111000000      33
000000100110000001      29
000000000000000001      26
000000100110100110      16
000000100110100111       3
Name: count, dtype: int64


### Make balanced dataset

In [ ]:
def create_balanced_dataset_simple(df, top_k_patterns=6, samples_per_pattern=100, random_state=42, shuffle=True):
    balanced_parts = []

    for pattern_id in range(top_k_patterns):
        group = df[df['pattern_id'] == pattern_id]
        if len(group) >= samples_per_pattern:
            sampled = group.sample(n=samples_per_pattern, random_state=random_state)
        else:
            sampled = group.sample(n=samples_per_pattern, replace=True, random_state=random_state)
        balanced_parts.append(sampled)

    balanced_df = pd.concat(balanced_parts, ignore_index=True)

    if shuffle:
        balanced_df = balanced_df.sample(frac=1.0, random_state=random_state).reset_index(drop=True)

    print(f"✅ Balanced dataset with {samples_per_pattern} samples for each of {top_k_patterns} patterns.")

    return balanced_df


In [84]:
df = pd.read_csv("D:/ML4pmg/Data_Large4/mesh_features_annotated.csv")
balanced_df = create_balanced_dataset_simple(df, top_k_patterns=8, samples_per_pattern=200)
balanced_df.to_csv("D:/ML4pmg/Data_Large4/balanced_mesh_features.csv", index=False)


✅ Balanced dataset with 200 samples for each of 8 patterns.


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  

See `MLmodels.ipynb` for RandomForest Classification and Regression

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  

In [87]:
df.columns



Index(['mesh_name', 'mesh_id', 'n_elements', 'n_nodes', 'n_fractures',
       'area_mean', 'area_median', 'area_std', 'area_5pct', 'area_95pct',
       ...
       'length_max', 'orientation_mean', 'orientation_median',
       'orientation_std', 'orientation_5pct', 'orientation_95pct',
       'orientation_min', 'orientation_max', 'convergence_pattern',
       'pattern_id'],
      dtype='object', length=113)